### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [364]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [365]:
# Create and find the necessary variables in the original dataframe
schools = len(school_data_complete['School ID'].drop_duplicates())
schools
students = len(school_data_complete['Student ID'].drop_duplicates())
students
budget = sum(school_data_complete['budget'].drop_duplicates())
budget
avg_math = school_data_complete['math_score'].mean()
avg_math
avg_reading = school_data_complete['reading_score'].mean()
avg_reading
# Calculate variables that require functions
avg_pass_math = school_data_complete['Student ID'][school_data_complete['math_score'] >= 70].count()/students*100
avg_pass_math
avg_pass_reading = school_data_complete['Student ID'][school_data_complete['reading_score'] >= 70].count()/students*100
avg_pass_reading
avg_pass = school_data_complete['Student ID'][school_data_complete['math_score'] >= 70][school_data_complete['reading_score'] >= 70].count()/students*100
avg_pass
# Create new dataframe
summary_df = pd.DataFrame({'Total Schools':[schools], 'Total Students':[students], 'Total Budget':[budget],
                           'Average Math Score':[avg_math], 'Average Reading Score':[avg_reading],
                           '% Passing Math':[avg_pass_math], '% Passing Reading':[avg_pass_reading],
                           '% Overall Passing':[avg_pass]})
# Format columns
summary_df["Total Students"] = summary_df["Total Students"].map('{0:,.0f}'.format)
summary_df["Total Budget"] = summary_df["Total Budget"].map('${0:,.2f}'.format)
summary_df["Average Math Score"] = summary_df["Average Math Score"].map('{0:,.2f}%'.format)
summary_df["Average Reading Score"] = summary_df["Average Reading Score"].map('{0:,.2f}%'.format)
summary_df["% Passing Math"] = summary_df["% Passing Math"].map('{0:,.2f}%'.format)
summary_df["% Passing Reading"] = summary_df["% Passing Reading"].map('{0:,.2f}%'.format)
summary_df["% Overall Passing"] = summary_df["% Overall Passing"].map('{0:,.2f}%'.format)
summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99%,81.88%,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [366]:
# Create groupby with necessary column
school_df = school_data_complete.groupby('school_name')
# Calculate varaible columns, rename and limit columns to necessary variables
## Type and size
school_type_df = school_df.max()
school_type_df = school_type_df[['type', 'size']] 
school_count_df = school_df.count()
school_count_df = school_count_df.rename(columns={'Student ID':'Total Students'})
school_count_df = school_count_df[['Total Students','type','size']]
## Averages
school_avg_df = school_df.mean()
school_avg_df = school_avg_df[['reading_score','math_score','budget']]
school_math = school_data_complete[school_data_complete.math_score >= 70]
school_math_gb = school_math.groupby('school_name')
school_math_df = school_math_gb.count()/school_count_df*100
school_math_df = school_math_df[['math_score', 'size']]
school_math_df = school_math_df.rename(columns={'size':'% Passing Math'})
school_reading = school_data_complete[school_data_complete.reading_score >= 70]
school_reading_gb = school_reading.groupby('school_name')
school_reading_df = school_reading_gb.count()/school_count_df*100
school_reading_df = school_reading_df[['reading_score', 'size']]
school_reading_df = school_reading_df.rename(columns={'size':'% Passing Reading'})
## Passing rates
school_pass = pd.merge(school_math, school_reading, on='Student ID')
school_pass_clean = school_pass[['Student ID', 'student_name_x', 'gender_x', 'grade_x', 'school_name_x',
                                 'reading_score_x', 'math_score_x', 'School ID_x', 'type_x', 'size_x', 'budget_x']]
school_pass_rename = school_pass_clean.rename(columns={'student_name_x':'student_name', 'gender_x':'gender',
                                                 'grade_x':'grade', 'school_name_x':'school_name',
                                                 'reading_score_x':'reading_score', 'math_score_x':'math_score',
                                                 'School ID_x':'School ID', 'type_x':'type', 'size_x':'size',
                                                 'budget_x':'budget'})
school_pass_gb = school_pass_rename.groupby('school_name')
school_pass_df = school_pass_gb.count()/school_count_df*100
school_pass_df = school_pass_df[['budget', 'size']]
school_pass_df = school_pass_df.rename(columns={'size':'% Overall Passing'})
school_count_df['Per Student Budget'] = school_avg_df.loc[:,'budget']/school_count_df.loc[:,'Total Students']
# Merge dataframes and clean columns
school_fin_df = pd.merge(school_count_df, school_avg_df, on='school_name')
school_fin_df = pd.merge(school_fin_df, school_math_df, on='school_name')
school_fin_df = pd.merge(school_fin_df, school_reading_df, on='school_name')
school_fin_df = pd.merge(school_fin_df, school_pass_df, on='school_name')
school_fin_df = pd.merge(school_fin_df, school_type_df, on='school_name')
school_summary_df = school_fin_df[['type_y', 'Total Students','budget_x',
                                   'Per Student Budget', 'math_score_x',
                                   'reading_score_x', '% Passing Math',
                                   '% Passing Reading', '% Overall Passing']]
school_summary_df = school_summary_df.rename(columns={'type_y':'School Type',
                                                      'budget_x':'Total School Budget',
                                                      'math_score_x':'Average Math Score',
                                                      'reading_score_x':'Average Reading Score'})
# Format columns
school_summary_df["Total Students"] = school_summary_df["Total Students"].map('{0:,.0f}'.format)
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map('${0:,.2f}'.format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map('${0:,.2f}'.format)
school_summary_df["Average Math Score"] = school_summary_df["Average Math Score"].map('{0:,.2f}%'.format)
school_summary_df["Average Reading Score"] = school_summary_df["Average Reading Score"].map('{0:,.2f}%'.format)
school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"].map('{0:,.2f}%'.format)
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"].map('{0:,.2f}%'.format)
school_summary_df["% Overall Passing"] = school_summary_df["% Overall Passing"].map('{0:,.2f}%'.format)
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05%,81.03%,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10%,80.75%,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29%,80.93%,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80%,83.81%,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63%,81.18%,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07%,80.97%,66.06%,81.22%,53.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [367]:
# Sort via % Overall Passing, High->Low
school_summary_top = school_summary_df.sort_values(by=['% Overall Passing'], ascending=False)
school_summary_top.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42%,83.85%,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27%,83.99%,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84%,84.04%,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [368]:
# Sort via % Overall Passing, Low->High
school_summary_bot = school_summary_df.sort_values(by=['% Overall Passing'], ascending=True)
school_summary_bot.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84%,80.74%,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63%,81.18%,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29%,80.93%,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07%,80.97%,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [369]:
# Create dataframe with only necessary columns
school_grades = school_data_complete[['school_name', 'grade', 'math_score', 'reading_score']]
# Split data per grade
school_math_9th = school_grades[school_grades.grade == '9th']
school_math_10th = school_grades[school_grades.grade == '10th']
school_math_11th = school_grades[school_grades.grade == '11th']
school_math_12th = school_grades[school_grades.grade == '12th']
# Calculate 9th grade average and group
school_math_9th_ser = school_math_9th[['school_name', 'math_score']]
school_math_9th_gb = school_math_9th_ser.groupby('school_name').mean()
school_math_9th_rn = school_math_9th_gb.rename(columns={'math_score':'9th'})
# Calculate 10th grade average and group
school_math_10th_ser = school_math_10th[['school_name', 'math_score']]
school_math_10th_gb = school_math_10th_ser.groupby('school_name').mean()
school_math_10th_rn = school_math_10th_gb.rename(columns={'math_score':'10th'})
# Calculate 11th grade average and group
school_math_11th_ser = school_math_11th[['school_name', 'math_score']]
school_math_11th_gb = school_math_11th_ser.groupby('school_name').mean()
school_math_11th_rn = school_math_11th_gb.rename(columns={'math_score':'11th'})
# Calculate 12th grade average and group
school_math_12th_ser = school_math_12th[['school_name', 'math_score']]
school_math_12th_gb = school_math_12th_ser.groupby('school_name').mean()
school_math_12th_rn = school_math_12th_gb.rename(columns={'math_score':'12th'})
# Merge dataframes
school_math_df = pd.merge(school_math_9th_rn, school_math_10th_rn, on='school_name')
school_math_df = pd.merge(school_math_df, school_math_11th_rn, on='school_name')
school_math_df = pd.merge(school_math_df, school_math_12th_rn, on='school_name')
# Format columns
school_math_df["9th"] = school_math_df["9th"].map('{0:,.2f}%'.format)
school_math_df["10th"] = school_math_df["10th"].map('{0:,.2f}%'.format)
school_math_df["11th"] = school_math_df["11th"].map('{0:,.2f}%'.format)
school_math_df["12th"] = school_math_df["12th"].map('{0:,.2f}%'.format)
school_math_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [370]:
# Split data per grade
school_read_9th = school_grades[school_grades.grade == '9th']
school_read_10th = school_grades[school_grades.grade == '10th']
school_read_11th = school_grades[school_grades.grade == '11th']
school_read_12th = school_grades[school_grades.grade == '12th']
# Calculate 9th grade average and group
school_read_9th_ser = school_read_9th[['school_name', 'reading_score']]
school_read_9th_gb = school_read_9th_ser.groupby('school_name').mean()
school_read_9th_rn = school_read_9th_gb.rename(columns={'reading_score':'9th'})
# Calculate 10th grade average and group
school_read_10th_ser = school_read_10th[['school_name', 'reading_score']]
school_read_10th_gb = school_read_10th_ser.groupby('school_name').mean()
school_read_10th_rn = school_read_10th_gb.rename(columns={'reading_score':'10th'})
# Calculate 11th grade average and group
school_read_11th_ser = school_read_11th[['school_name', 'reading_score']]
school_read_11th_gb = school_read_11th_ser.groupby('school_name').mean()
school_read_11th_rn = school_read_11th_gb.rename(columns={'reading_score':'11th'})
# Calculate 12th grade average and group
school_read_12th_ser = school_read_12th[['school_name', 'reading_score']]
school_read_12th_gb = school_read_12th_ser.groupby('school_name').mean()
school_read_12th_rn = school_read_12th_gb.rename(columns={'reading_score':'12th'})
# Merge dataframes
school_read_df = pd.merge(school_read_9th_rn, school_read_10th_rn, on='school_name')
school_read_df = pd.merge(school_read_df, school_read_11th_rn, on='school_name')
school_read_df = pd.merge(school_read_df, school_read_12th_rn, on='school_name')
# Format columns
school_read_df["9th"] = school_read_df["9th"].map('{0:,.2f}%'.format)
school_read_df["10th"] = school_read_df["10th"].map('{0:,.2f}%'.format)
school_read_df["11th"] = school_read_df["11th"].map('{0:,.2f}%'.format)
school_read_df["12th"] = school_read_df["12th"].map('{0:,.2f}%'.format)
school_read_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [371]:
# Create bins
spend_bins = [0, 584.9, 629.9, 644.9, 675]
spend_groups = ['<$584', '$585-629', '$630-644', '$645-675']
# Pull necessary columns from original dataframe
spend_df = school_data_complete.loc[:,['budget', 'Student ID', 'math_score', 'reading_score', 'size']]
# Calculate varaible columns, rename and limit columns to necessary variables
spend_df['Spending (Per student)'] = spend_df.loc[:,'budget']/spend_df.loc[:,'size']
spend_df['Pass Math'] = spend_df['math_score'][spend_df.math_score >= 70]
spend_df['Pass Reading'] = spend_df['reading_score'][spend_df.reading_score >= 70]
spend_df['Pass Overall'] = spend_df['Student ID'][spend_df.reading_score >= 70][spend_df.math_score >= 70]
spend_fin_df = spend_df
# Group via bins and clean
spend_fin_df['Spending Ranges (Per student)'] = pd.cut(spend_df['Spending (Per student)'], spend_bins, labels=spend_groups, include_lowest=False)
spend_mean_gb = spend_fin_df.groupby('Spending Ranges (Per student)').mean()
spend_mean_rn = spend_mean_gb[['math_score', 'reading_score']]
spend_mean_df = spend_mean_rn.rename(columns={'math_score':'Average Math Score', 'reading_score':'Average Reading Score'})
spend_count_gb = spend_fin_df.groupby('Spending Ranges (Per student)').count()
spend_count_gb['% Passing Math'] = (spend_count_gb['Pass Math']/spend_count_gb['math_score'])*100
spend_count_gb['% Passing Reading'] = (spend_count_gb['Pass Reading']/spend_count_gb['reading_score'])*100
spend_count_gb['% Overall Passing'] = (spend_count_gb['Pass Overall']/spend_count_gb['Student ID'])*100
spend_count_df = spend_count_gb[['% Passing Math', '% Passing Reading', '% Overall Passing']]
# Merge dataframes
spend_summary_df = pd.merge(spend_mean_df, spend_count_df, on='Spending Ranges (Per student)')
# Format columns
spend_summary_df['Average Math Score'] = spend_summary_df['Average Math Score'].map('{0:,.2f}%'.format)
spend_summary_df['Average Reading Score'] = spend_summary_df['Average Reading Score'].map('{0:,.2f}%'.format)
spend_summary_df['% Passing Math'] = spend_summary_df['% Passing Math'].map('{0:,.2f}%'.format)
spend_summary_df['% Passing Reading'] = spend_summary_df['% Passing Reading'].map('{0:,.2f}%'.format)
spend_summary_df['% Overall Passing'] = spend_summary_df['% Overall Passing'].map('{0:,.2f}%'.format)
spend_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per student),,,,,
<$584,83.36%,83.96%,93.70%,96.69%,90.64%
$585-629,79.98%,82.31%,79.11%,88.51%,70.94%
$630-644,77.82%,81.30%,70.62%,82.60%,58.84%
$645-675,77.05%,81.01%,66.23%,81.11%,53.53%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [372]:
# Create bins
size_bins = [0, 1000, 2000, 5000]
size_groups = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
# Pull necessary columns from original dataframe
size_df = school_data_complete.loc[:,['Student ID', 'math_score', 'reading_score', 'size']]
size_df['Pass Math'] = size_df['math_score'][size_df.math_score >= 70]
size_df['Pass Reading'] = size_df['reading_score'][size_df.reading_score >= 70]
size_df['Pass Overall'] = size_df['Student ID'][size_df.reading_score >= 70][size_df.math_score >= 70]
size_fin_df = size_df
# Group via bins and clean
size_fin_df['School Size'] = pd.cut(size_df['size'], size_bins, labels=size_groups, include_lowest=False)
size_mean_gb = size_fin_df.groupby('School Size').mean()
size_mean_rn = size_mean_gb[['math_score', 'reading_score']]
size_mean_df = size_mean_rn.rename(columns={'math_score':'Average Math Score', 'reading_score':'Average Reading Score'})
size_count_gb = size_fin_df.groupby('School Size').count()
size_count_gb['% Passing Math'] = (size_count_gb['Pass Math']/size_count_gb['math_score'])*100
size_count_gb['% Passing Reading'] = (size_count_gb['Pass Reading']/size_count_gb['reading_score'])*100
size_count_gb['% Overall Passing'] = (size_count_gb['Pass Overall']/size_count_gb['Student ID'])*100
size_count_df = size_count_gb[['% Passing Math', '% Passing Reading', '% Overall Passing']]
# Merge dataframes
size_summary_df = pd.merge(size_mean_df, size_count_df, on='School Size')
# Format columns
size_summary_df['Average Math Score'] = size_summary_df['Average Math Score'].map('{0:,.2f}%'.format)
size_summary_df['Average Reading Score'] = size_summary_df['Average Reading Score'].map('{0:,.2f}%'.format)
size_summary_df['% Passing Math'] = size_summary_df['% Passing Math'].map('{0:,.2f}%'.format)
size_summary_df['% Passing Reading'] = size_summary_df['% Passing Reading'].map('{0:,.2f}%'.format)
size_summary_df['% Overall Passing'] = size_summary_df['% Overall Passing'].map('{0:,.2f}%'.format)
size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.83%,83.97%,93.95%,96.04%,90.14%
Medium (1000-2000),83.37%,83.87%,93.62%,96.77%,90.62%
Large (2000-5000),77.48%,81.20%,68.65%,82.13%,56.57%


## Scores by School Type

* Perform the same operations as above, based on school type

In [373]:
# Pull necessary columns from original dataframe
type_df = school_data_complete.loc[:,['Student ID', 'math_score', 'reading_score', 'size', 'type']]
type_df['Pass Math'] = type_df['math_score'][type_df.math_score >= 70]
type_df['Pass Reading'] = type_df['reading_score'][type_df.reading_score >= 70]
type_df['Pass Overall'] = type_df['Student ID'][type_df.reading_score >= 70][size_df.math_score >= 70]
type_df = type_df.rename(columns={'type':'School Type'})
type_fin_df = type_df
# Group by School type and calculate functions
type_mean_gb = type_fin_df.groupby('School Type').mean()
type_mean_rn = type_mean_gb[['math_score', 'reading_score']]
type_mean_df = type_mean_rn.rename(columns={'math_score':'Average Math Score', 'reading_score':'Average Reading Score'})
type_count_gb = type_fin_df.groupby('School Type').count()
type_count_gb['% Passing Math'] = (type_count_gb['Pass Math']/type_count_gb['math_score'])*100
type_count_gb['% Passing Reading'] = (type_count_gb['Pass Reading']/type_count_gb['reading_score'])*100
type_count_gb['% Overall Passing'] = (type_count_gb['Pass Overall']/type_count_gb['Student ID'])*100
type_count_df = type_count_gb[['% Passing Math', '% Passing Reading', '% Overall Passing']]
# Merge dataframes
type_summary_df = pd.merge(type_mean_df, type_count_df, on='School Type')
# Format columns
type_summary_df['Average Math Score'] = type_summary_df['Average Math Score'].map('{0:,.2f}%'.format)
type_summary_df['Average Reading Score'] = type_summary_df['Average Reading Score'].map('{0:,.2f}%'.format)
type_summary_df['% Passing Math'] = type_summary_df['% Passing Math'].map('{0:,.2f}%'.format)
type_summary_df['% Passing Reading'] = type_summary_df['% Passing Reading'].map('{0:,.2f}%'.format)
type_summary_df['% Overall Passing'] = type_summary_df['% Overall Passing'].map('{0:,.2f}%'.format)
type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.41%,83.90%,93.70%,96.65%,90.56%
District,76.99%,80.96%,66.52%,80.91%,53.70%
